# 数据预处理
使用了所有提供的数据集，共10个。

 完成解压


In [ ]:
import os
import zipfile
import shutil
from tqdm import tqdm

# 创建dataset目录（如果不存在）
dataset_dir = 'dataset'
os.makedirs(dataset_dir, exist_ok=True)

# 原始数据集路径
original_dir = 'original_dataset'

print("开始解压并整理数据集...")
for zip_name in tqdm(os.listdir(original_dir)):
    if zip_name.endswith('.zip'):
        # 提取项目名称（去掉.zip后缀）
        project_name = zip_name[:-4]
        project_dir = os.path.join(dataset_dir, project_name)
        
        # 创建项目子目录
        os.makedirs(project_dir, exist_ok=True)
        
        # 解压文件
        zip_path = os.path.join(original_dir, zip_name)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(project_dir)
        
        # 查找并处理嵌套的项目目录
        nested_dirs = [d for d in os.listdir(project_dir) 
                      if os.path.isdir(os.path.join(project_dir, d)) 
                      and not d.startswith('__MACOSX')]
        
        if nested_dirs:
            # 假设第一个非__MACOSX目录是项目目录
            nested_project_dir = os.path.join(project_dir, nested_dirs[0])
            
            # 移动所有文件到顶级项目目录
            for item in os.listdir(nested_project_dir):
                src = os.path.join(nested_project_dir, item)
                dst = os.path.join(project_dir, item)
                
                # 如果是文件则移动，如果是目录则移动整个目录
                if os.path.isfile(src):
                    shutil.move(src, dst)
                else:
                    shutil.move(src, project_dir)
            
            # 删除嵌套的项目目录
            shutil.rmtree(nested_project_dir)
        
        # 删除__MACOSX目录（如果存在）
        macosx_dir = os.path.join(project_dir, '__MACOSX')
        if os.path.exists(macosx_dir):
            shutil.rmtree(macosx_dir)
        
        print(f"已整理: {zip_name} -> {project_dir}")

print("所有数据集解压并整理完成！")
print(f"整理后的数据集位于: {os.path.abspath(dataset_dir)}")

## 预览前几行数据

In [ ]:
import pandas as pd
import os
import json

# 选择django项目作为示例
project_name = "django"
project_dir = os.path.join("dataset", project_name)

print(f"预览项目: {project_name}")
print("="*50)

# 列出所有文件
files = os.listdir(project_dir)
print("可用文件列表:")
for i, f in enumerate(files, 1):
    print(f"{i}. {f}")
print("="*50)

# 预览每个Excel文件
excel_files = [f for f in files if f.endswith('.xlsx')]
for file in excel_files:
    file_path = os.path.join(project_dir, file)
    print(f"\n文件: {file}")
    print("-"*50)
    
    try:
        # 获取所有sheet名称
        sheets = pd.ExcelFile(file_path).sheet_names
        
        if len(sheets) > 0:
            sheet_name = sheets[0]
            df = pd.read_excel(file_path, sheet_name=sheet_name)
            
            print(f"Sheet名称: {sheet_name}")
            print(f"行数: {df.shape[0]}, 列数: {df.shape[1]}")
            
            # 打印列名
            print("\n列名:")
            print(", ".join(df.columns.tolist()))
            
            # 打印前1行数据
            print("\n前1行数据:")
            for i in range(min(1, len(df))):
                row = df.iloc[i]
                print(f"行 {i+1}:")
                for col in df.columns:
                    if col=="comment_embedding":
                        print(f"  {col}: {row[col]}")
                    else:
                        print(f"  {col}: {str(row[col])[0:10]}")
        else:
            print("警告: 没有找到任何sheet")
    except Exception as e:
        print(f"读取文件出错: {str(e)}")
    
    print("="*50)

# 预览txt文件
# txt_files = [f for f in files if f.endswith('.txt')]
# for file in txt_files:
#     file_path = os.path.join(project_dir, file)
#     print(f"\n文件: {file}")
#     print("-"*50)
#     
#     try:
#         with open(file_path, 'r') as f:
#             # 尝试读取前5行
#             lines = []
#             for _ in range(5):
#                 try:
#                     lines.append(next(f).strip())
#                 except StopIteration:
#                     break
#             
#             print("前5行内容:")
#             for i, line in enumerate(lines, 1):
#                 print(f"{i}. {line}")
#                 
#             # 如果是JSON格式，尝试解析
#             if file == "pr_time_dict.txt":
#                 print("\n尝试解析为JSON:")
#                 f.seek(0)
#                 try:
#                     data = json.load(f)
#                     print(f"数据类型: {type(data)}")
#                     print(f"键数量: {len(data)}")
#                     
#                     # 显示前3个键值对
#                     print("\n前3个键值对:")
#                     for i, (k, v) in enumerate(data.items()):
#                         if i >= 3:
#                             break
#                         print(f"{k}: {v}")
#                 except json.JSONDecodeError:
#                     print("无法解析为JSON格式")
#     except Exception as e:
#         print(f"读取文件出错: {str(e)}")
#     
#     print("="*50)

# 以下数据处理分为三步完成

## 第一步 数据合并

## 第二步 数据清洗

## 第三步 特征工程

## 第一步 数据合并


修复pr_time_dict.txt的格式，方便作为json处理

In [ ]:
import os
import json
import ast

dataset_dir = "dataset"
projects = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

print("开始修复所有项目的 pr_time_dict.txt 文件...")
print("="*50)

for project in projects:
    file_path = os.path.join(dataset_dir, project, "pr_time_dict.txt")
    
    try:
        print(f"处理项目: {project}")
        
        # 1. 读取文件内容
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        
        # 2. 特殊处理：替换 nan 为 None
        content = content.replace('nan', 'None')
        
        # 3. 使用 ast.literal_eval 解析
        time_dict = ast.literal_eval(content)
        
        # 4. 转换 None 为 null（JSON格式）
        # 递归函数处理嵌套字典
        def convert_none(obj):
            if isinstance(obj, dict):
                return {k: convert_none(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_none(item) for item in obj]
            elif obj is None:
                return None  # JSON 中 None 会转为 null
            else:
                return obj
        
        time_dict = convert_none(time_dict)
        
        # 5. 保存为标准JSON格式
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(time_dict, f, indent=2)
        
        print(f"  成功修复并保存: {project}/pr_time_dict.txt")
        print("-"*50)
    
    except Exception as e:
        print(f"  无法修复 {project}/pr_time_dict.txt: {str(e)}")
        # 备份原始文件
        backup_path = file_path + ".bak"
        with open(backup_path, 'w', encoding='utf-8') as f:
            f.write(content)
        print(f"  已创建备份文件: {backup_path}")
        print("-"*50)

print("\n所有项目修复完成!")

In [1]:
import os
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

# 创建输出目录
output_dir = "processed_dataset/merged"
os.makedirs(output_dir, exist_ok=True)

# 获取所有项目列表
dataset_dir = "dataset"
projects = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

print(f"开始合并项目数据，共 {len(projects)} 个项目")
print("="*50)

# 使用tqdm创建进度条
for project in tqdm(projects, desc="处理项目"):
    project_dir = os.path.join(dataset_dir, project)
    
    print(f"\n处理项目: {project}")
    print("-"*50)
    
    try:
        # 1. 读取基础表 - PR_info_add_conversation.xlsx
        base_df = pd.read_excel(os.path.join(project_dir, "PR_info_add_conversation.xlsx"))
        print(f"  基础表读取成功，行数: {len(base_df)}")
        
        # 2. 合并PR_features.xlsx
        try:
            pr_features = pd.read_excel(os.path.join(project_dir, "PR_features.xlsx"))
            base_df = pd.merge(base_df, pr_features, on="number", how="left", suffixes=('', '_pr'))
            print(f"  合并PR特征，新增列数: {len(pr_features.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法合并PR特征 - {str(e)}")
        
        # 3. 合并author_features.xlsx
        try:
            author_features = pd.read_excel(os.path.join(project_dir, "author_features.xlsx"))
            base_df = pd.merge(base_df, author_features, on="number", how="left", suffixes=('', '_author'))
            print(f"  合并作者特征，新增列数: {len(author_features.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法合并作者特征 - {str(e)}")
        
        # 4. 合并reviewer_features.xlsx
        try:
            reviewer_features = pd.read_excel(os.path.join(project_dir, "reviewer_features.xlsx"))
            base_df = pd.merge(base_df, reviewer_features, on="number", how="left", suffixes=('', '_reviewer'))
            print(f"  合并评审者特征，新增列数: {len(reviewer_features.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法合并评审者特征 - {str(e)}")
        
        # 5. 合并project_features.xlsx
        try:
            project_features = pd.read_excel(os.path.join(project_dir, "project_features.xlsx"))
            # 项目特征对所有PR相同，直接添加列
            for col in project_features.columns:
                if col != "number":
                    base_df[col] = project_features[col].iloc[0]
            print(f"  合并项目特征，新增列数: {len(project_features.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法合并项目特征 - {str(e)}")
        
        # 6. 处理pr_time_dict.txt
        try:
            with open(os.path.join(project_dir, "pr_time_dict.txt"), 'r', encoding='utf-8') as f:
                time_dict = json.load(f)
            
            # 创建时间特征DataFrame
            time_df = pd.DataFrame.from_dict(time_dict, orient='index')
            time_df.index = time_df.index.astype(int)  # 确保索引为整数
            time_df.reset_index(inplace=True)
            time_df.rename(columns={'index': 'number'}, inplace=True)
            
            # 合并到主表
            base_df = pd.merge(base_df, time_df, on="number", how="left", suffixes=('', '_time'))
            print(f"  合并时间字典，新增列数: {len(time_df.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法处理时间字典 - {str(e)}")
        
        # 7. 处理评论信息（聚合） - 修复错误
        try:
            comment_df = pd.read_excel(os.path.join(project_dir, "PR_comment_info.xlsx"))
            print(f"  读取评论信息，行数: {len(comment_df)}")
            
            # 修复错误：确保body列是字符串类型
            if 'body' in comment_df.columns:
                # 将非字符串转换为字符串
                comment_df['body'] = comment_df['body'].apply(lambda x: str(x) if not pd.isna(x) else "")
            
            # 计算每个PR的聚合特征
            comment_agg = comment_df.groupby('belong_to_PR').agg(
                comment_count=pd.NamedAgg(column='id', aggfunc='count'),
                last_comment_time=pd.NamedAgg(column='created_at', aggfunc='max')
            ).reset_index()
            
            # 安全地计算平均评论长度
            if 'body' in comment_df.columns:
                # 计算每个评论的长度
                comment_df['body_length'] = comment_df['body'].str.len()
                # 计算每个PR的平均评论长度
                avg_length = comment_df.groupby('belong_to_PR')['body_length'].mean().reset_index()
                avg_length.rename(columns={'body_length': 'avg_comment_length'}, inplace=True)
                # 合并到聚合表
                comment_agg = pd.merge(comment_agg, avg_length, on='belong_to_PR', how='left')
            
            comment_agg.rename(columns={'belong_to_PR': 'number'}, inplace=True)
            
            # 合并到主表
            base_df = pd.merge(base_df, comment_agg, on="number", how="left")
            print(f"  添加评论聚合特征，新增列数: {len(comment_agg.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法处理评论信息 - {str(e)}")
        
        # 8. 处理提交信息（聚合）
        try:
            commit_df = pd.read_excel(os.path.join(project_dir, "PR_commit_info.xlsx"))
            print(f"  读取提交信息，行数: {len(commit_df)}")
            
            # 修复可能的错误：确保file_name_list是字符串
            if 'file_name_list' in commit_df.columns:
                commit_df['file_name_list'] = commit_df['file_name_list'].astype(str)
            
            # 计算每个PR的聚合特征
            commit_agg = commit_df.groupby('belong_to_PR').agg(
                commit_count=pd.NamedAgg(column='sha', aggfunc='count'),
                total_additions=pd.NamedAgg(column='additions', aggfunc='sum'),
                total_deletions=pd.NamedAgg(column='deletions', aggfunc='sum'),
                total_files_changed=pd.NamedAgg(column='file_name_list', aggfunc=lambda x: x.nunique())
            ).reset_index()
            commit_agg.rename(columns={'belong_to_PR': 'number'}, inplace=True)
            
            # 合并到主表
            base_df = pd.merge(base_df, commit_agg, on="number", how="left")
            print(f"  添加提交聚合特征，新增列数: {len(commit_agg.columns)-1}")
        except Exception as e:
            print(f"  警告: 无法处理提交信息 - {str(e)}")
        
        # 保存合并后的数据集
        output_path = os.path.join(output_dir, f"{project}_merged.xlsx")
        base_df.to_excel(output_path, index=False)
        print(f"  合并完成! 总列数: {len(base_df.columns)}")
        print(f"  已保存到: {output_path}")
        
    except Exception as e:
        print(f"  处理失败: {str(e)}")
    
    print("="*50)

print("\n所有项目处理完成!")
print(f"合并后的数据集保存在: {os.path.abspath(output_dir)}")

开始合并项目数据，共 10 个项目


处理项目:   0%|          | 0/10 [00:00<?, ?it/s]


处理项目: django
--------------------------------------------------
  基础表读取成功，行数: 16976
  合并PR特征，新增列数: 34
  合并作者特征，新增列数: 15
  合并评审者特征，新增列数: 16
  合并项目特征，新增列数: 30
  合并时间字典，新增列数: 4
  读取评论信息，行数: 46204
  添加评论聚合特征，新增列数: 3
  读取提交信息，行数: 32888
  添加提交聚合特征，新增列数: 4


处理项目:  10%|█         | 1/10 [00:55<08:17, 55.27s/it]

  合并完成! 总列数: 115
  已保存到: processed_dataset/merged\django_merged.xlsx

处理项目: moby
--------------------------------------------------
  基础表读取成功，行数: 23515
  合并PR特征，新增列数: 34
  合并作者特征，新增列数: 15
  合并评审者特征，新增列数: 16
  合并项目特征，新增列数: 30
  合并时间字典，新增列数: 4
  读取评论信息，行数: 133170
  添加评论聚合特征，新增列数: 3
  读取提交信息，行数: 41736
  添加提交聚合特征，新增列数: 4


...

  合并完成! 总列数: 115
  已保存到: processed_dataset/merged\tensorflow_merged.xlsx

处理项目: terraform
--------------------------------------------------
  基础表读取成功，行数: 13219
  合并PR特征，新增列数: 34
  合并作者特征，新增列数: 15
  合并评审者特征，新增列数: 16
  合并项目特征，新增列数: 30
  合并时间字典，新增列数: 4
  读取评论信息，行数: 42128
  添加评论聚合特征，新增列数: 3
  读取提交信息，行数: 31384
  添加提交聚合特征，新增列数: 4


处理项目:  90%|█████████ | 9/10 [10:50<01:11, 71.78s/it]

  合并完成! 总列数: 115
  已保存到: processed_dataset/merged\terraform_merged.xlsx

处理项目: yii2
--------------------------------------------------
  基础表读取成功，行数: 8040
  合并PR特征，新增列数: 34
  合并作者特征，新增列数: 15
  合并评审者特征，新增列数: 16
  合并项目特征，新增列数: 30
  合并时间字典，新增列数: 4
  读取评论信息，行数: 25643
  添加评论聚合特征，新增列数: 3
  读取提交信息，行数: 23581
  添加提交聚合特征，新增列数: 4


处理项目: 100%|██████████| 10/10 [11:18<00:00, 67.89s/it]

  合并完成! 总列数: 115
  已保存到: processed_dataset/merged\yii2_merged.xlsx

所有项目处理完成!
合并后的数据集保存在: D:\hehaochuan\codes\machine_learning\lab2\processed_dataset\merged


# 第二步 数据清洗

预览合并后数据结构

In [2]:
import pandas as pd
import os

# 选择要预览的项目
project_name = "django"  # 替换为您想预览的项目名称

# 合并数据文件路径
merged_file = os.path.join("processed_dataset", "merged", f"{project_name}_merged.xlsx")

print(f"预览项目: {project_name}")
print("="*50)

try:
    # 读取Excel文件
    df = pd.read_excel(merged_file)
    
    # 显示基本信息
    print(f"文件路径: {merged_file}")
    print(f"总行数: {len(df)}")
    print(f"总列数: {len(df.columns)}")
    print("="*50)
    
    # 显示所有列名
    print("\n所有列名:")
    print("-"*50)
    for i, col in enumerate(df.columns, 1):
        print(f"{i}. {col}")
    
    print("="*50)
    
    # 显示第一行数据
    print("\n第一行数据:")
    print("-"*50)
    
    # 获取第一行数据
    first_row = df.iloc[0]
    
    # 按列分组显示
    groups = {
        "基本信息": ["number", "state", "title", "author", "merged", "conversation", "closed_at"],
        "时间信息": ["created_at", "updated_at", "merged_at", "closed_at", "last_comment_time"],
        "变更统计": ["additions", "deletions", "changed_files", "commit_count", "comment_count"],
        "技术特征": ["has_test", "has_feature", "has_bug", "has_document", "file_type"],
        "作者特征": ["experience", "participation", "change_num", "merge_proportion"],
        "评审者特征": ["avg_comments", "avg_round", "avg_duration"],
        "项目特征": ["project_age", "team_size", "changes_per_week"],
        "聚合特征": ["total_additions", "total_deletions", "total_files_changed", "avg_comment_length"]
    }
    
    # 显示每组的关键列
    for group_name, columns in groups.items():
        print(f"\n{group_name}组:")
        print("-"*20)
        
        # 只显示该组中实际存在的列
        existing_columns = [col for col in columns if col in df.columns]
        
        for col in existing_columns:
            value = first_row[col]
            
            # 处理长文本和嵌入向量
            if isinstance(value, str) and len(value) > 100:
                value = value[:100] + "..."
            elif isinstance(value, list):
                value = f"列表[{len(value)}项]"
            
            print(f"  {col}: {value}")
    
    print("="*50)
    
    # 保存列名到文件以便详细查看
    columns_file = os.path.join("processed_dataset", f"{project_name}_columns.txt")
    with open(columns_file, 'w', encoding='utf-8') as f:
        f.write(f"{project_name} 数据集列名列表 (共{len(df.columns)}列):\n")
        f.write("="*50 + "\n")
        for i, col in enumerate(df.columns, 1):
            f.write(f"{i}. {col}\n")
    
    print(f"\n完整列名列表已保存到: {columns_file}")
    print("="*50)

except Exception as e:
    print(f"预览失败: {str(e)}")
    print("="*50)

预览项目: django
文件路径: processed_dataset\merged\django_merged.xlsx
总行数: 16976
总列数: 115

所有列名:
--------------------------------------------------
1. number
2. state
3. title
4. author
5. body
6. created_at
7. updated_at
8. merged_at
9. merged
10. comments
11. review_comments
12. commits
13. additions
14. deletions
15. changed_files
16. conversation
17. closed_at
18. directory_num
19. language_num
20. file_type
21. has_test
22. has_feature
23. has_bug
24. has_document
25. has_improve
26. has_refactor
27. title_length
28. title_readability
29. title_embedding
30. body_length
31. body_readability
32. body_embedding
33. lines_added
34. lines_deleted
35. segs_added
36. segs_deleted
37. segs_updated
38. files_added
39. files_deleted
40. files_updated
41. modify_proportion
42. modify_entropy
43. test_churn
44. non_test_churn
45. reviewer_num
46. bot_reviewer_num
47. is_reviewed
48. comment_num
49. comment_length
50. comment_embedding
51. last_comment_mention
52. name
53. experience
54. is_reviewer

统计缺失值占比和重复率（pr）

In [6]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm

# 创建统计结果目录
stats_dir = "processed_dataset/stats"
os.makedirs(stats_dir, exist_ok=True)

# 获取所有项目列表
dataset_dir = "dataset"
projects = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

print(f"开始对所有项目进行统计分析，共 {len(projects)} 个项目")
print("="*50)

# 创建汇总数据框
summary_missing = pd.DataFrame()
summary_duplicates = pd.DataFrame(columns=["Project", "Duplicate_Count", "Duplicate_Percentage"])

# 遍历所有项目
for project in tqdm(projects, desc="分析项目"):
    project_dir = os.path.join(dataset_dir, project)
    merged_file = os.path.join("processed_dataset", "merged", f"{project}_merged.xlsx")
    
    print(f"\n分析项目: {project}")
    print("-"*50)
    
    try:
        # 读取Excel文件
        df = pd.read_excel(merged_file)
        
        # 显示基本信息
        print(f"  文件路径: {merged_file}")
        print(f"  总行数: {len(df)}")
        print(f"  总列数: {len(df.columns)}")
        
        # 1. 缺失值统计
        missing_data = df.isnull().sum().reset_index()
        missing_data.columns = ['Column', 'Missing_Count']
        missing_data['Missing_Percentage'] = (missing_data['Missing_Count'] / len(df)) * 100
        
        # 添加到汇总表
        missing_data['Project'] = project
        summary_missing = pd.concat([summary_missing, missing_data], ignore_index=True)
        
        # 打印高缺失率列
        high_missing = missing_data[missing_data['Missing_Percentage'] > 10]
        if len(high_missing) > 0:
            print(f"  高缺失率列 (>10%):")
            for _, row in high_missing.iterrows():
                print(f"    {row['Column']}: {row['Missing_Percentage']:.2f}%")
        else:
            print("  没有高缺失率列 (>10%)")
        
        # 2. 重复值统计
        duplicate_counts = df['number'].duplicated().sum()
        duplicate_percentage = (duplicate_counts / len(df)) * 100
        
        # 添加到汇总表 - 修复警告问题
        new_row = pd.DataFrame({
            "Project": [project],
            "Duplicate_Count": [duplicate_counts],
            "Duplicate_Percentage": [duplicate_percentage]
        })
        
        summary_duplicates = pd.concat([summary_duplicates, new_row], ignore_index=True)
        
        print(f"  PR number重复数量: {duplicate_counts} ({duplicate_percentage:.2f}%)")
        
        # 3. 保存项目统计结果
        project_stats_dir = os.path.join(stats_dir, project)
        os.makedirs(project_stats_dir, exist_ok=True)
        
        # 保存缺失值统计
        missing_file = os.path.join(project_stats_dir, f"{project}_missing_stats.csv")
        missing_data.to_csv(missing_file, index=False)
        print(f"  缺失值统计已保存到: {missing_file}")
        
        # 保存重复值详情
        if duplicate_counts > 0:
            duplicates = df[df['number'].duplicated(keep=False)]
            duplicates_file = os.path.join(project_stats_dir, f"{project}_duplicates.csv")
            duplicates.to_csv(duplicates_file, index=False)
            print(f"  重复值详情已保存到: {duplicates_file}")
        
        print("-"*50)
    
    except Exception as e:
        print(f"  分析失败: {str(e)}")
        print("-"*50)

# 保存汇总统计结果
print("\n保存汇总统计结果")
print("="*50)

# 缺失值汇总
summary_missing_file = os.path.join(stats_dir, "all_projects_missing_summary.csv")
summary_missing.to_csv(summary_missing_file, index=False)
print(f"所有项目缺失值汇总已保存到: {summary_missing_file}")

# 重复值汇总
summary_duplicates_file = os.path.join(stats_dir, "all_projects_duplicates_summary.csv")
summary_duplicates.to_csv(summary_duplicates_file, index=False)
print(f"所有项目重复值汇总已保存到: {summary_duplicates_file}")

# 高缺失率列分析
if not summary_missing.empty:
    high_missing_summary = summary_missing[summary_missing['Missing_Percentage'] > 10]
    if not high_missing_summary.empty:
        high_missing_file = os.path.join(stats_dir, "high_missing_columns.csv")
        high_missing_summary.to_csv(high_missing_file, index=False)
        print(f"高缺失率列详情已保存到: {high_missing_file}")
        
        # 按列分组分析
        column_missing_stats = high_missing_summary.groupby('Column')['Missing_Percentage'].agg(['mean', 'max', 'min'])
        column_missing_stats.columns = ['Average_Missing', 'Max_Missing', 'Min_Missing']
        column_missing_stats = column_missing_stats.sort_values(by='Average_Missing', ascending=False)
        
        column_stats_file = os.path.join(stats_dir, "column_missing_stats.csv")
        column_missing_stats.to_csv(column_stats_file)
        print(f"列缺失率统计已保存到: {column_stats_file}")
        
        print("\n高缺失率列统计:")
        print(column_missing_stats.head(10))
    else:
        print("没有高缺失率列 (>10%)")
else:
    print("缺失值汇总为空，无法进行高缺失率列分析")

print("\n所有项目统计分析完成!")
print("="*50)

开始对所有项目进行统计分析，共 10 个项目


分析项目:   0%|          | 0/10 [00:00<?, ?it/s]


分析项目: django
--------------------------------------------------


C:\Users\hehaochuan\AppData\Local\Temp\ipykernel_146012\925333143.py:67: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  summary_duplicates = pd.concat([summary_duplicates, new_row], ignore_index=True)
分析项目:  10%|█         | 1/10 [00:13<01:58, 13.22s/it]

  文件路径: processed_dataset\merged\django_merged.xlsx
  总行数: 16976
  总列数: 115
  高缺失率列 (>10%):
    body: 21.65%
    merged_at: 51.44%
    name_reviewer: 23.29%
    merged_at_time: 51.44%
    comment_count: 23.29%
    last_comment_time: 23.29%
    avg_comment_length: 23.29%
  PR number重复数量: 0 (0.00%)
  缺失值统计已保存到: processed_dataset/stats\django\django_missing_stats.csv
--------------------------------------------------

分析项目: moby
--------------------------------------------------


...

  文件路径: processed_dataset\merged\tensorflow_merged.xlsx
  总行数: 23130
  总列数: 115
  高缺失率列 (>10%):
    body: 17.66%
    merged_at: 30.63%
    name_reviewer: 33.23%
    merged_at_time: 30.63%
    comment_count: 33.23%
    last_comment_time: 33.23%
    avg_comment_length: 33.23%
  PR number重复数量: 0 (0.00%)
  缺失值统计已保存到: processed_dataset/stats\tensorflow\tensorflow_missing_stats.csv
--------------------------------------------------

分析项目: terraform
--------------------------------------------------


分析项目:  90%|█████████ | 9/10 [02:28<00:15, 15.50s/it]

  文件路径: processed_dataset\merged\terraform_merged.xlsx
  总行数: 13219
  总列数: 115
  高缺失率列 (>10%):
    merged_at: 18.47%
    merged_at_time: 18.47%
  PR number重复数量: 0 (0.00%)
  缺失值统计已保存到: processed_dataset/stats\terraform\terraform_missing_stats.csv
--------------------------------------------------

分析项目: yii2
--------------------------------------------------


分析项目: 100%|██████████| 10/10 [02:34<00:00, 15.42s/it]

  文件路径: processed_dataset\merged\yii2_merged.xlsx
  总行数: 8040
  总列数: 115
  高缺失率列 (>10%):
    body: 20.85%
    merged_at: 36.58%
    merged_at_time: 36.58%
  PR number重复数量: 0 (0.00%)
  缺失值统计已保存到: processed_dataset/stats\yii2\yii2_missing_stats.csv
--------------------------------------------------

保存汇总统计结果
所有项目缺失值汇总已保存到: processed_dataset/stats\all_projects_missing_summary.csv
所有项目重复值汇总已保存到: processed_dataset/stats\all_projects_duplicates_summary.csv
高缺失率列详情已保存到: processed_dataset/stats\high_missing_columns.csv
列缺失率统计已保存到: processed_dataset/stats\column_missing_stats.csv

高缺失率列统计:
                    Average_Missing  Max_Missing  Min_Missing
Column                                                       
merged_at_time            28.772269    51.437323    12.386931
merged_at                 28.772269    51.437323    12.386931
avg_comment_length        23.150916    39.274824    12.435082
comment_count             23.150916    39.274824    12.435082
name_reviewer             23.150916    3

## 完成数据清洗

### 特殊说明
对于state是open，则merge_at,closed_at,update_at等允许缺失而不进行填充

### 删除重复记录：
按 number列删除重复的PR记录
保留最新的记录（基于 updated_at）
### 删除 merged_at_time 列：
检测并删除 merged_at_time,closed_at_time...列（与 merged_at...重复）
### 统一时间格式：
将时间列转换为 datetime 格式
使用 fix_time_format函数处理各种时间格式
确保时间顺序正确（created_at≤ updated_at≤ closed_at）
### 处理 closed_at 缺失值：
用 updated_at填充 closed_at
如果 updated_at也缺失，则删除该行记录
### 处理其他缺失值：
高缺失率列采用特定策略填充
数值列用中位数填充
时间列用前后值填充
分类列用众数填充
文本列用空字符串填充
### 处理异常值：
时间异常：确保时间顺序正确
数值异常：负值设为0，极端值进行缩尾处理
### 数据类型转换：
布尔列转换为0/1整数
时间列统一为datetime类型
### 文本清洗
移除HTML标签和特殊字符
截断过长的文本
### 生成清洗报告：
记录清洗前后的记录数变化
记录每列的缺失值处理情况
保存为文本文件

In [2]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import re
import json
from pandas.api.types import is_datetime64_any_dtype, is_datetime64tz_dtype, DatetimeTZDtype

def clean_project_data(df):
    """
    清洗单个项目的数据，并返回清洗后的DataFrame和统计信息
    """
    # 初始化统计信息字典
    stats = {
        "original_rows": len(df),
        "rows_removed": 0,
        "columns_processed": {},
        "filled_values": 0,
        "removed_values": 0
    }
    
    # 1. 删除重复记录
    original_rows = len(df)
    df = df.sort_values('updated_at', ascending=False)
    df = df.drop_duplicates('number', keep='first')
    stats["rows_removed"] += original_rows - len(df)
    
    # 2. 删除重复的时间列
    # 这些列与 created_at, updated_at, closed_at 重复
    duplicate_time_cols = [
        'created_at_time', 'updated_at_time', 'closed_at_time', 'merged_at_time'
    ]
    
    for col in duplicate_time_cols:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
            stats["columns_processed"][col] = {"action": "dropped", "count": 1}
            print(f"  已删除重复列: {col}")
    
    # 3. 统一时间格式
    # 定义需要处理的时间列
    time_cols = ['created_at', 'updated_at', 'closed_at', 'merged_at', 'last_comment_time']
    
    for col in time_cols:
        if col in df.columns:
            # 初始化列统计
            if col not in stats["columns_processed"]:
                stats["columns_processed"][col] = {"filled": 0, "removed": 0}
                
            # 记录原始缺失值数量
            original_missing = df[col].isna().sum()
            
            # 尝试转换为datetime格式
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except:
                # 如果转换失败，尝试修复格式
                df[col] = df[col].apply(lambda x: fix_time_format(x) if pd.notnull(x) else x)
                df[col] = pd.to_datetime(df[col], errors='coerce')
            
            # 记录转换后的缺失值
            new_missing = df[col].isna().sum()
            if new_missing > original_missing:
                stats["columns_processed"][col]["filled"] += (new_missing - original_missing)
                stats["filled_values"] += (new_missing - original_missing)
    
    # 4. 处理 closed_at 缺失值 - 只处理非open状态的PR
    if 'closed_at' in df.columns and 'state' in df.columns:
        if 'closed_at' not in stats["columns_processed"]:
            stats["columns_processed"]["closed_at"] = {"filled": 0, "removed": 0}
            
        # 只处理非open状态的PR
        non_open_mask = df['state'] != 'open'
        
        # 记录原始缺失值数量
        original_missing = df.loc[non_open_mask, 'closed_at'].isna().sum()
        
        # 用 updated_at 填充 closed_at（仅限非open状态）
        mask = non_open_mask & df['closed_at'].isna() & df['updated_at'].notna()
        filled_count = mask.sum()
        df.loc[mask, 'closed_at'] = df.loc[mask, 'updated_at']
        
        # 记录填充数量
        stats["columns_processed"]["closed_at"]["filled"] += filled_count
        stats["filled_values"] += filled_count
        
        # 删除仍缺失的行（仅限非open状态）
        mask_to_drop = non_open_mask & df['closed_at'].isna()
        initial_count = len(df)
        df = df[~mask_to_drop]
        removed_count = initial_count - len(df)
        
        # 记录删除数量
        stats["columns_processed"]["closed_at"]["removed"] += removed_count
        stats["removed_values"] += removed_count
        stats["rows_removed"] += removed_count
    
    # 5. 处理其他缺失值
    high_missing_cols = {
        'merged_at': pd.NaT,
        'avg_comment_length': 0,
        'comment_count': 0,
        'name_reviewer': 'Unknown',
        'last_comment_time': df['created_at'],
        'modify_entropy': df['modify_entropy'].median(),
        'body': 'null',  # 修改为'null'
        'title': 'null'   # 添加title字段的填充
    }
    
    for col, fill_value in high_missing_cols.items():
        if col in df.columns:
            # 初始化列统计
            if col not in stats["columns_processed"]:
                stats["columns_processed"][col] = {"filled": 0, "removed": 0}
                
            # 记录原始缺失值数量
            original_missing = df[col].isna().sum()
            
            # 填充缺失值
            if isinstance(fill_value, pd.Series):
                df[col] = df[col].fillna(fill_value)
            else:
                df[col] = df[col].fillna(fill_value)
            
            # 记录填充数量
            filled_count = original_missing - df[col].isna().sum()
            stats["columns_processed"][col]["filled"] += filled_count
            stats["filled_values"] += filled_count
    
    # 其他列处理
    for col in df.columns:
        if col in stats["columns_processed"]:
            continue  # 已处理过的列跳过
            
        if df[col].isna().sum() > 0:
            # 初始化列统计
            stats["columns_processed"][col] = {"filled": 0, "removed": 0}
            
            # 记录原始缺失值数量
            original_missing = df[col].isna().sum()
            
            if pd.api.types.is_numeric_dtype(df[col]):
                # 数值列用中位数填充
                df[col] = df[col].fillna(df[col].median())
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                # 时间列不填充，保留缺失值
                pass  # 不填充，保留NaN
            else:
                # 分类列用众数填充
                mode_value = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
                df[col] = df[col].fillna(mode_value)
            
            # 记录填充数量
            filled_count = original_missing - df[col].isna().sum()
            stats["columns_processed"][col]["filled"] += filled_count
            stats["filled_values"] += filled_count
    
    # 6. 处理异常值
    # 数值异常处理
    numeric_cols = df.select_dtypes(include=np.number).columns
    for col in numeric_cols:
        # 负值设为0
        negative_count = (df[col] < 0).sum()
        df[col] = np.where(df[col] < 0, 0, df[col])
        
        # 记录处理数量
        if negative_count > 0:
            if col not in stats["columns_processed"]:
                stats["columns_processed"][col] = {"filled": 0, "removed": 0}
            stats["columns_processed"][col]["filled"] += negative_count
            stats["filled_values"] += negative_count
        
        # 处理极端值（Winsorize）
        q1 = df[col].quantile(0.01)
        q99 = df[col].quantile(0.99)
        low_outliers = (df[col] < q1).sum()
        high_outliers = (df[col] > q99).sum()
        
        df[col] = np.where(df[col] < q1, q1, df[col])
        df[col] = np.where(df[col] > q99, q99, df[col])
        
        # 记录处理数量
        if low_outliers > 0 or high_outliers > 0:
            if col not in stats["columns_processed"]:
                stats["columns_processed"][col] = {"filled": 0, "removed": 0}
            stats["columns_processed"][col]["filled"] += (low_outliers + high_outliers)
            stats["filled_values"] += (low_outliers + high_outliers)
    
    # 7. 数据类型转换
    # 扩展布尔列列表
    bool_cols = [
        'merged', 'has_test', 'has_feature', 'has_bug', 
        'has_document', 'has_improve', 'has_refactor', 
        'is_reviewed', 'last_comment_mention', 'is_reviewer', 'is_author'
    ]
    
    for col in bool_cols:
        if col in df.columns:
            # 记录原始非数值数量
            non_numeric_count = (~df[col].apply(lambda x: isinstance(x, (int, float)))).sum()
            
            df[col] = df[col].astype(int)
            
            # 记录转换数量
            if non_numeric_count > 0:
                if col not in stats["columns_processed"]:
                    stats["columns_processed"][col] = {"filled": 0, "removed": 0}
                stats["columns_processed"][col]["filled"] += non_numeric_count
                stats["filled_values"] += non_numeric_count
    
    # 8. 文本清洗
    if 'body' in df.columns:
        # 记录原始非字符串数量
        non_string_count = (~df['body'].apply(lambda x: isinstance(x, str))).sum()
        
        df['body'] = df['body'].apply(clean_text)
        
        # 确保body字段没有缺失值（最终检查）
        df['body'] = df['body'].fillna('null')  # 修改为'null'
        
        # 记录处理数量
        if non_string_count > 0:
            if 'body' not in stats["columns_processed"]:
                stats["columns_processed"]['body'] = {"filled": 0, "removed": 0}
            stats["columns_processed"]['body']["filled"] += non_string_count
            stats["filled_values"] += non_string_count
    
    if 'title' in df.columns:
        # 记录原始非字符串数量
        non_string_count = (~df['title'].apply(lambda x: isinstance(x, str))).sum()
        
        df['title'] = df['title'].apply(clean_text)
        
        # 确保title字段没有缺失值（最终检查）
        df['title'] = df['title'].fillna('null')  # 修改为'null'
        
        # 记录处理数量
        if non_string_count > 0:
            if 'title' not in stats["columns_processed"]:
                stats["columns_processed"]['title'] = {"filled": 0, "removed": 0}
            stats["columns_processed"]['title']["filled"] += non_string_count
            stats["filled_values"] += non_string_count
    
    # 添加最终行数
    stats["final_rows"] = len(df)
    
    # 移除时间列的时区信息（解决Excel不支持时区的问题）
    for col in time_cols:
        if col in df.columns:
            # 使用新的方法检查时区类型
            if isinstance(df[col].dtype, pd.DatetimeTZDtype):
                df[col] = df[col].dt.tz_localize(None)
            # 或者转换为无时区的datetime
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = pd.to_datetime(df[col]).dt.tz_localize(None)
    
    return df, stats

def fix_time_format(time_str):
    """
    修复时间格式问题
    """
    if pd.isnull(time_str):
        return time_str
    
    # 尝试解析常见格式
    try:
        # ISO 8601格式 (YYYY-MM-DDTHH:MM:SSZ)
        if 'T' in time_str and 'Z' in time_str:
            return pd.to_datetime(time_str, utc=True)
        
        # 日期时间格式 (YYYY-MM-DD HH:MM:SS)
        if ' ' in time_str and len(time_str) > 10:
            return pd.to_datetime(time_str)
        
        # 纯日期格式 (YYYY-MM-DD)
        if len(time_str) == 10 and '-' in time_str:
            return pd.to_datetime(time_str)
        
        # 其他格式尝试解析
        return pd.to_datetime(time_str)
    except:
        return pd.NaT

def clean_text(text):
    """
    清洗文本数据
    """
    if pd.isnull(text):
        return "null"  # 修改为'null'
    
    # 确保是字符串
    if not isinstance(text, str):
        text = str(text)
    
    # 移除HTML标签
    text = re.sub(r'<[^>]*>', '', text)
    # 移除特殊字符
    text = re.sub(r'[^\w\s.,!?;:\'"-]', '', text)
    # 截断长文本
    return text[:500]

# 自定义JSON编码器，处理numpy类型
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, pd.Timestamp):
            return str(obj)
        else:
            return super(NumpyEncoder, self).default(obj)

# 主程序
if __name__ == "__main__":
    # 创建清洗结果目录
    cleaned_dir = "processed_dataset/cleaned"
    stats_dir = "processed_dataset/stats"
    os.makedirs(cleaned_dir, exist_ok=True)
    os.makedirs(stats_dir, exist_ok=True)
    
    # 获取所有项目列表
    dataset_dir = "dataset"
    projects = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    
    print(f"开始清洗所有项目数据，共 {len(projects)} 个项目")
    print("="*50)
    
    # 初始化总统计
    total_stats = {
        "projects": [],
        "total_original_rows": 0,
        "total_final_rows": 0,
        "total_rows_removed": 0,
        "total_filled_values": 0,
        "total_removed_values": 0
    }
    
    # 遍历所有项目
    for project in tqdm(projects, desc="清洗项目"):
        merged_file = os.path.join("processed_dataset", "merged", f"{project}_merged.xlsx")
        cleaned_file = os.path.join(cleaned_dir, f"{project}_cleaned.xlsx")
        stats_file = os.path.join(stats_dir, f"{project}_stats.json")
        
        print(f"\n清洗项目: {project}")
        print(f"  输入文件: {merged_file}")
        
        try:
            # 读取合并后的数据
            df = pd.read_excel(merged_file)
            print(f"  原始记录数: {len(df)}")
            print(f"  原始列数: {len(df.columns)}")
            
            # 更新总统计
            total_stats["total_original_rows"] += len(df)
            
            # 清洗数据
            df_cleaned, stats = clean_project_data(df)
            print(f"  清洗后记录数: {len(df_cleaned)}")
            print(f"  清洗后列数: {len(df_cleaned.columns)}")
            
            # 更新总统计
            total_stats["projects"].append(project)
            total_stats["total_final_rows"] += len(df_cleaned)
            total_stats["total_rows_removed"] += stats["rows_removed"]
            total_stats["total_filled_values"] += stats["filled_values"]
            total_stats["total_removed_values"] += stats["removed_values"]
            
            # 保存清洗后的数据
            # 确保所有时间列都是无时区的
            time_cols = [
                'created_at', 'updated_at', 'closed_at', 'merged_at', 'last_comment_time'
            ]
            for col in time_cols:
                if col in df_cleaned.columns and pd.api.types.is_datetime64_any_dtype(df_cleaned[col]):
                    df_cleaned[col] = df_cleaned[col].dt.tz_localize(None)
            
            # 确保body和title字段没有缺失值
            if 'body' in df_cleaned.columns:
                df_cleaned['body'] = df_cleaned['body'].fillna('null')
            if 'title' in df_cleaned.columns:
                df_cleaned['title'] = df_cleaned['title'].fillna('null')
            
            df_cleaned.to_excel(cleaned_file, index=False)
            print(f"  已保存到: {cleaned_file}")
            
            # 保存统计信息 - 使用自定义编码器处理numpy类型
            with open(stats_file, 'w') as f:
                json.dump(stats, f, indent=4, cls=NumpyEncoder)
            print(f"  统计信息已保存到: {stats_file}")
            
            # 打印详细统计
            print(f"  删除行数: {stats['rows_removed']}")
            print(f"  填充值数: {stats['filled_values']}")
            print(f"  删除值数: {stats['removed_values']}")
            print(f"  删除列数: {sum(1 for col_stats in stats['columns_processed'].values() if col_stats.get('action') == 'dropped')}")
            print("-"*50)
            
        except Exception as e:
            print(f"  清洗失败: {str(e)}")
            print("-"*50)
    
    # 保存总统计 - 使用自定义编码器处理numpy类型
    total_stats_file = os.path.join(stats_dir, "total_stats.json")
    with open(total_stats_file, 'w') as f:
        json.dump(total_stats, f, indent=4, cls=NumpyEncoder)
    
    print("\n所有项目数据清洗完成!")
    print("="*50)
    print(f"总原始行数: {total_stats['total_original_rows']}")
    print(f"总最终行数: {total_stats['total_final_rows']}")
    print(f"总删除行数: {total_stats['total_rows_removed']}")
    print(f"总填充值数: {total_stats['total_filled_values']}")
    print(f"总删除值数: {total_stats['total_removed_values']}")
    print("="*50)
    print(f"总统计信息已保存到: {total_stats_file}")

开始清洗所有项目数据，共 10 个项目


清洗项目:   0%|          | 0/10 [00:00<?, ?it/s]


清洗项目: django
  输入文件: processed_dataset\merged\django_merged.xlsx
  原始记录数: 16976
  原始列数: 115
  已删除重复列: created_at_time
  已删除重复列: updated_at_time
  已删除重复列: closed_at_time
  已删除重复列: merged_at_time
  清洗后记录数: 16976
  清洗后列数: 111


清洗项目:  10%|█         | 1/10 [01:04<09:38, 64.32s/it]

  已保存到: processed_dataset/cleaned\django_cleaned.xlsx
  统计信息已保存到: processed_dataset/stats\django_stats.json
  删除行数: 0
  填充值数: 30566
  删除值数: 0
  删除列数: 4
--------------------------------------------------

清洗项目: moby
  输入文件: processed_dataset\merged\moby_merged.xlsx
  原始记录数: 23515
  原始列数: 115
  已删除重复列: created_at_time
  已删除重复列: updated_at_time
  已删除重复列: closed_at_time
  已删除重复列: merged_at_time
  清洗后记录数: 23515
  清洗后列数: 111


...

  已保存到: processed_dataset/cleaned\tensorflow_cleaned.xlsx
  统计信息已保存到: processed_dataset/stats\tensorflow_stats.json
  删除行数: 0
  填充值数: 52326
  删除值数: 0
  删除列数: 4
--------------------------------------------------

清洗项目: terraform
  输入文件: processed_dataset\merged\terraform_merged.xlsx
  原始记录数: 13219
  原始列数: 115
  已删除重复列: created_at_time
  已删除重复列: updated_at_time
  已删除重复列: closed_at_time
  已删除重复列: merged_at_time
  清洗后记录数: 13219
  清洗后列数: 111


清洗项目:  90%|█████████ | 9/10 [11:49<01:16, 76.99s/it]

  已保存到: processed_dataset/cleaned\terraform_cleaned.xlsx
  统计信息已保存到: processed_dataset/stats\terraform_stats.json
  删除行数: 0
  填充值数: 11493
  删除值数: 0
  删除列数: 4
--------------------------------------------------

清洗项目: yii2
  输入文件: processed_dataset\merged\yii2_merged.xlsx
  原始记录数: 8040
  原始列数: 115
  已删除重复列: created_at_time
  已删除重复列: updated_at_time
  已删除重复列: closed_at_time
  已删除重复列: merged_at_time
  清洗后记录数: 8040
  清洗后列数: 111


清洗项目: 100%|██████████| 10/10 [12:19<00:00, 73.93s/it]

  已保存到: processed_dataset/cleaned\yii2_cleaned.xlsx
  统计信息已保存到: processed_dataset/stats\yii2_stats.json
  删除行数: 0
  填充值数: 11115
  删除值数: 0
  删除列数: 4
--------------------------------------------------

所有项目数据清洗完成!
总原始行数: 197627
总最终行数: 197627
总删除行数: 0
总填充值数: 333454
总删除值数: 0
总统计信息已保存到: processed_dataset/stats\total_stats.json


# 第三步 特征工程

### 时间特征：

processing_time：PR从创建到关闭的总时间（小时）

first_response_time：首次评论响应时间（小时）

last_response_time：最后评论响应时间（小时）

created_hour：创建时间的小时（0-23）

created_dayofweek：创建时间的星期几（0=周一，6=周日）

created_month：创建时间的月份（1-12）

### 文本特征：

title_length：标题长度（字符数）

body_length：正文长度（字符数）

has_bug_keyword：是否包含bug相关关键词

has_feature_keyword：是否包含功能相关关键词

has_document_keyword：是否包含文档相关关键词

### 变更特征：

total_changes：总变更行数（添加+删除）

net_changes：净变更行数（添加-删除）

change_density：变更密度（每文件变更行数）

additions_per_file：每文件添加行数

### 参与者特征：

author_experience：作者经验（项目内首次提交到当前PR的天数）

author_activity：作者活跃度（项目内PR数量）

reviewer_count：评审者数量（估算）

### 项目特征：

project_age：项目年龄（从第一个PR到当前PR的天数）

open_pr_count：开放PR数量（项目内）

### 交互特征：

author_exp_change_size：作者经验×变更大小

complexity_per_reviewer：每个评审者的变更复杂度

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import re

# 创建特征工程目录
engineered_dir = "processed_dataset/engineered"
os.makedirs(engineered_dir, exist_ok=True)

# 获取所有项目列表
cleaned_dir = "processed_dataset/cleaned"
projects = [f.replace('_cleaned.xlsx', '') for f in os.listdir(cleaned_dir) if f.endswith('_cleaned.xlsx')]

print(f"开始特征工程，共 {len(projects)} 个项目")
print("="*50)

# 关键词列表
bug_keywords = ['bug', 'fix', 'error', 'issue', 'defect', 'fault']
feature_keywords = ['feature', 'enhance', 'new', 'improve', 'add', 'implement']
document_keywords = ['doc', 'readme', 'document', 'comment', 'note']

def extract_features(df):
    """
    从清洗后的数据中提取特征
    """
    # 记录原始列数
    original_cols = set(df.columns)
    
    # 打印原始列名用于调试
    print(f"  原始列: {list(original_cols)}")
    
    # 1. 时间特征
    # 确保时间列是datetime类型
    time_cols = ['created_at', 'closed_at', 'last_comment_time']
    for col in time_cols:
        if col in df.columns:
            print(f"  处理时间列: {col}")
            # 确保列存在且不为空
            if df[col].isna().any():
                # 填充缺失值为当前时间
                df[col] = df[col].fillna(pd.Timestamp.now())
            try:
                df[col] = pd.to_datetime(df[col])
                print(f"    成功转换为datetime类型")
            except Exception as e:
                print(f"    转换失败: {str(e)}")
    
    # 时间差特征 - 处理缺失值
    if 'closed_at' in df.columns and 'created_at' in df.columns:
        print("  计算处理时间")
        # 确保没有缺失值
        mask = df['closed_at'].notna() & df['created_at'].notna()
        df.loc[mask, 'processing_time'] = (df.loc[mask, 'closed_at'] - df.loc[mask, 'created_at']).dt.total_seconds() / 3600
        # 填充缺失值
        df['processing_time'] = df['processing_time'].fillna(-1)  # 用-1表示缺失
        print(f"    添加特征: processing_time")
    
    if 'last_comment_time' in df.columns and 'created_at' in df.columns:
        print("  计算最后响应时间")
        mask = df['last_comment_time'].notna() & df['created_at'].notna()
        df.loc[mask, 'last_response_time'] = (df.loc[mask, 'last_comment_time'] - df.loc[mask, 'created_at']).dt.total_seconds() / 3600
        df['last_response_time'] = df['last_response_time'].fillna(-1)
        print(f"    添加特征: last_response_time")
    
    # 时间分解特征 - 处理缺失值
    if 'created_at' in df.columns:
        print("  分解创建时间")
        # 确保没有缺失值
        df['created_at'] = df['created_at'].fillna(pd.Timestamp.now())
        df['created_hour'] = df['created_at'].dt.hour
        df['created_dayofweek'] = df['created_at'].dt.dayofweek  # 0=周一, 6=周日
        df['created_month'] = df['created_at'].dt.month
        print(f"    添加特征: created_hour, created_dayofweek, created_month")
    
    # 2. 文本特征 - 处理缺失值
    if 'title' in df.columns:
        print("  处理标题特征")
        df['title'] = df['title'].fillna('null')
        df['title_length'] = df['title'].str.len()
        print(f"    添加特征: title_length")
    
    if 'body' in df.columns:
        print("  处理正文特征")
        df['body'] = df['body'].fillna('null')
        df['body_length'] = df['body'].str.len()
        print(f"    添加特征: body_length")
        
        # 转换为小写以便匹配
        body_lower = df['body'].str.lower().fillna('null')
        
        # 定义关键词列表
        bug_keywords = ['bug', 'fix', 'error', 'issue', 'defect', 'fault']
        feature_keywords = ['feature', 'enhance', 'new', 'improve', 'add', 'implement']
        document_keywords = ['doc', 'readme', 'document', 'comment', 'note']
        
        # 关键词特征 - 处理缺失值
        print("  添加关键词特征")
        for keyword_list, col_name in zip([bug_keywords, feature_keywords, document_keywords], 
                                         ['has_bug_keyword', 'has_feature_keyword', 'has_document_keyword']):
            # 使用apply安全处理
            df[col_name] = body_lower.apply(
                lambda x: 1 if any(kw in x for kw in keyword_list) else 0
            )
            print(f"    添加特征: {col_name}")
    
    # 3. 变更特征 - 处理缺失值
    if 'additions' in df.columns and 'deletions' in df.columns:
        print("  处理变更特征")
        # 填充缺失值
        df['additions'] = df['additions'].fillna(0)
        df['deletions'] = df['deletions'].fillna(0)
        df['total_changes'] = df['additions'] + df['deletions']
        df['net_changes'] = df['additions'] - df['deletions']
        print(f"    添加特征: total_changes, net_changes")
    
    if 'changed_files' in df.columns:
        print("  处理文件变更特征")
        # 填充缺失值并避免除以0
        df['changed_files'] = df['changed_files'].fillna(1)
        changed_files = df['changed_files'].replace(0, 1)
        
        if 'total_changes' in df.columns:
            df['change_density'] = df['total_changes'] / changed_files
            print(f"    添加特征: change_density")
        
        if 'additions' in df.columns:
            df['additions_per_file'] = df['additions'] / changed_files
            print(f"    添加特征: additions_per_file")
    
    # 4. 参与者特征 - 处理缺失值
    if 'author' in df.columns:
        print("  处理作者特征")
        # 确保作者列没有缺失值
        df['author'] = df['author'].fillna('Unknown')
        
        # 作者经验（项目内首次提交到当前PR的天数）
        author_min_date = df.groupby('author')['created_at'].transform('min')
        df['author_experience'] = (df['created_at'] - author_min_date).dt.days.fillna(0)
        print(f"    添加特征: author_experience")
        
        # 作者活跃度（项目内PR数量）
        df['author_activity'] = df.groupby('author')['number'].transform('count').fillna(0)
        print(f"    添加特征: author_activity")
    
    # 使用'review_comments'列估算评审者数量
    if 'review_comments' in df.columns:
        print("  处理评审者特征")
        # 简单估算评审者数量（每个评论视为一个评审者）
        # 安全处理非列表类型
        df['reviewer_count'] = df['review_comments'].apply(
            lambda x: len(x) if isinstance(x, list) else 1 if pd.notna(x) else 0
        )
        print(f"    添加特征: reviewer_count")
    
    # 5. 项目特征 - 处理缺失值
    if 'project' in df.columns and 'created_at' in df.columns:
        print("  处理项目特征")
        # 确保项目列没有缺失值
        df['project'] = df['project'].fillna('Unknown')
        
        # 项目年龄（从第一个PR到当前PR的天数）
        project_min_date = df.groupby('project')['created_at'].transform('min')
        df['project_age'] = (df['created_at'] - project_min_date).dt.days.fillna(0)
        print(f"    添加特征: project_age")
        
        # 开放PR数量（项目内）
        open_pr_count = df.groupby('project')['state'].transform(lambda x: (x == 'open').sum())
        df['open_pr_count'] = open_pr_count.fillna(0)
        print(f"    添加特征: open_pr_count")
    
    # 6. 交互特征 - 处理缺失值
    if 'author_experience' in df.columns and 'total_changes' in df.columns:
        print("  添加交互特征")
        # 填充缺失值
        df['author_experience'] = df['author_experience'].fillna(0)
        df['total_changes'] = df['total_changes'].fillna(0)
        df['author_exp_change_size'] = df['author_experience'] * df['total_changes']
        print(f"    添加特征: author_exp_change_size")
    
    if 'reviewer_count' in df.columns and 'total_changes' in df.columns:
        # 填充缺失值
        df['reviewer_count'] = df['reviewer_count'].fillna(0)
        df['total_changes'] = df['total_changes'].fillna(0)
        df['complexity_per_reviewer'] = df['total_changes'] / (df['reviewer_count'].replace(0, 1))  # 避免除以0
        print(f"    添加特征: complexity_per_reviewer")
    
    # 计算新增特征数
    new_cols = set(df.columns) - original_cols
    print(f"  新增特征数: {len(new_cols)}")
    print(f"  新增特征列表: {list(new_cols)}")
    
    return df

# 遍历所有项目
for project in tqdm(projects, desc="特征工程"):
    cleaned_file = os.path.join(cleaned_dir, f"{project}_cleaned.xlsx")
    engineered_file = os.path.join(engineered_dir, f"{project}_engineered.xlsx")
    
    print(f"\n处理项目: {project}")
    print(f"  输入文件: {cleaned_file}")
    
    try:
        # 读取清洗后的数据
        df = pd.read_excel(cleaned_file)
        print(f"  原始记录数: {len(df)}")
        
        # 特征工程
        df_engineered = extract_features(df)
        print(f"  新增特征数: {len(df_engineered.columns) - len(df.columns)}")
        
        # 保存特征工程后的数据
        df_engineered.to_excel(engineered_file, index=False)
        print(f"  已保存到: {engineered_file}")
        print("-"*50)
        
    except Exception as e:
        print(f"  特征工程失败: {str(e)}")
        print("-"*50)

print("\n所有项目特征工程完成!")
print("="*50)
print(f"特征工程后的数据集保存在: {engineered_dir}")

开始特征工程，共 10 个项目


特征工程:   0%|          | 0/10 [00:00<?, ?it/s]


处理项目: django
  输入文件: processed_dataset/cleaned\django_cleaned.xlsx
  原始记录数: 16976
  原始列: ['segs_updated', 'avg_segs', 'merge_proportion_reviewer', 'comments', 'file_type', 'comment_length', 'author_num', 'closeness_centrality', 'eigenvector_centrality', 'state', 'name', 'betweenness_centrality', 'avg_churn_abandoned', 'avg_files_abandoned', 'modify_proportion', 'test_churn', 'name_reviewer', 'merged', 'changes_per_reviewer', 'total_additions', 'directory_num', 'title_length', 'is_author', 'segs_deleted', 'merge_proportion', 'avg_duration_reviewer', 'change_num', 'language_num', 'avg_round_reviewer', 'avg_duration', 'add_per_week', 'updated_at', 'participation_reviewer', 'title', 'commit_count', 'body_readability', 'avg_rounds_abandoned', 'degree_centrality', 'is_reviewer', 'avg_lines', 'body', 'last_comment_time', 'clustering_coefficient', 'project_age', 'experience_reviewer', 'avg_rounds', 'changed_files', 'avg_comments_abandoned', 'participation', 'betweenness_centrality_reviewer', 

特征工程:  10%|█         | 1/10 [01:04<09:44, 64.99s/it]

  已保存到: processed_dataset/engineered\django_engineered.xlsx
--------------------------------------------------

处理项目: moby
  输入文件: processed_dataset/cleaned\moby_cleaned.xlsx
  原始记录数: 23515
  原始列: ['segs_updated', 'avg_segs', 'merge_proportion_reviewer', 'comments', 'file_type', 'comment_length', 'author_num', 'closeness_centrality', 'eigenvector_centrality', 'state', 'name', 'betweenness_centrality', 'avg_churn_abandoned', 'avg_files_abandoned', 'modify_proportion', 'test_churn', 'name_reviewer', 'merged', 'changes_per_reviewer', 'total_additions', 'directory_num', 'title_length', 'is_author', 'segs_deleted', 'merge_proportion', 'avg_duration_reviewer', 'change_num', 'language_num', 'avg_round_reviewer', 'avg_duration', 'add_per_week', 'updated_at', 'participation_reviewer', 'title', 'commit_count', 'body_readability', 'avg_rounds_abandoned', 'degree_centrality', 'is_reviewer', 'avg_lines', 'body', 'last_comment_time', 'clustering_coefficient', 'project_age', 'experience_reviewer', 'a

...

  已保存到: processed_dataset/engineered\tensorflow_engineered.xlsx
--------------------------------------------------

处理项目: terraform
  输入文件: processed_dataset/cleaned\terraform_cleaned.xlsx
  原始记录数: 13219
  原始列: ['segs_updated', 'avg_segs', 'merge_proportion_reviewer', 'comments', 'file_type', 'comment_length', 'author_num', 'closeness_centrality', 'eigenvector_centrality', 'state', 'name', 'betweenness_centrality', 'avg_churn_abandoned', 'avg_files_abandoned', 'modify_proportion', 'test_churn', 'name_reviewer', 'merged', 'changes_per_reviewer', 'total_additions', 'directory_num', 'title_length', 'is_author', 'segs_deleted', 'merge_proportion', 'avg_duration_reviewer', 'change_num', 'language_num', 'avg_round_reviewer', 'avg_duration', 'add_per_week', 'updated_at', 'participation_reviewer', 'title', 'commit_count', 'body_readability', 'avg_rounds_abandoned', 'degree_centrality', 'is_reviewer', 'avg_lines', 'body', 'last_comment_time', 'clustering_coefficient', 'project_age', 'experience

特征工程:  90%|█████████ | 9/10 [14:06<01:24, 84.00s/it]

  已保存到: processed_dataset/engineered\terraform_engineered.xlsx
--------------------------------------------------

处理项目: yii2
  输入文件: processed_dataset/cleaned\yii2_cleaned.xlsx
  原始记录数: 8040
  原始列: ['segs_updated', 'avg_segs', 'merge_proportion_reviewer', 'comments', 'file_type', 'comment_length', 'author_num', 'closeness_centrality', 'eigenvector_centrality', 'state', 'name', 'betweenness_centrality', 'avg_churn_abandoned', 'avg_files_abandoned', 'modify_proportion', 'test_churn', 'name_reviewer', 'merged', 'changes_per_reviewer', 'total_additions', 'directory_num', 'title_length', 'is_author', 'segs_deleted', 'merge_proportion', 'avg_duration_reviewer', 'change_num', 'language_num', 'avg_round_reviewer', 'avg_duration', 'add_per_week', 'updated_at', 'participation_reviewer', 'title', 'commit_count', 'body_readability', 'avg_rounds_abandoned', 'degree_centrality', 'is_reviewer', 'avg_lines', 'body', 'last_comment_time', 'clustering_coefficient', 'project_age', 'experience_reviewer', 

特征工程: 100%|██████████| 10/10 [14:37<00:00, 87.73s/it]

  已保存到: processed_dataset/engineered\yii2_engineered.xlsx
--------------------------------------------------

所有项目特征工程完成!
特征工程后的数据集保存在: processed_dataset/engineered
